코드 수정일시 : 2022.08.09 (v1.1) by nrkim

In [ ]:
#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


read_data 함수 : 데이터 파일에서 한줄씩 문자열을 읽어들인 후 Tab 문자를 기준으로 분리하여 반환한다.

In [ ]:
import codecs
def read_data(filename):
    with codecs.open(filename, 'r', 'utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()] #한줄씩 문자열 읽기
        data = data[1:] # 파일의 첫번째 라인을 제외하고 반환
    return data

# 데이터 파일 경로 확인하세요
train_data = read_data('/content/gdrive/My Drive/Colab Notebooks/aivle/data/nsm/small_ratings_train.txt')
test_data = read_data('/content/gdrive/My Drive/Colab Notebooks/aivle/data/nsm/small_ratings_test.txt')

print("학습데이터 크기:", len(train_data))
print("테스트데이터 크기:", len(test_data))
print (train_data[0])
print (test_data[0])

학습데이터 크기: 10000
테스트데이터 크기: 100
['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0']
['7898805', '음악이 주가 된, 최고의 음악영화', '1']


In [ ]:
# train_docs_X, test_docs_X : train data와 test data의 문장 저장
# train_Y, test_Y : train data와 test data의 정답 레이블 저장 (긍정:1 부정:0)
train_docs_X = [row[1] for row in train_data]
train_Y = [row[2] for row in train_data]

test_docs_X = [row[1] for row in test_data]
test_Y = [row[2] for row in test_data]

In [ ]:
print(train_docs_X[0])
print(train_Y[0])
print(test_docs_X[0])
print(test_Y[0])

원작의 긴장감을 제대로 살려내지못했다.
0
음악이 주가 된, 최고의 음악영화
1


#### CountVectorizer 클래스
tokenization과 occurrence counting을 수행

데이터 : ['I am happy', 'I am sad']<br>
단어 모음 : ['I', 'am', 'happy', 'sad']<br>
변환 결과 : [[1, 1, 1, 0], [1, 1, 0, 1]]<br>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(max_features = 1000).fit(train_docs_X)

train_X = vec.transform(train_docs_X).toarray()
test_X = vec.transform(test_docs_X).toarray()

In [ ]:
print(train_X[0])
print("학습데이터 크기:", len(train_X))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

###Naive Bayes 모델 이용하기

In [ ]:
%%time
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(train_X, train_Y)

print("Accuracy on training set: {:.3f}".format(gnb.score(train_X, train_Y)))
print("Accuracy on test set: {:.3f}".format(gnb.score(test_X, test_Y)))


Accuracy on training set: 0.708
Accuracy on test set: 0.640
CPU times: user 240 ms, sys: 37.1 ms, total: 277 ms
Wall time: 310 ms


In [ ]:
pos = [ [test_docs_X[i], test_Y[i], gnb.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '1' ]
neg = [ [test_docs_X[i], test_Y[i], gnb.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '0' ]

print ('>긍정 리뷰에 대한 예측:')
for i in pos[:5]:
    print ('\n입력', i[0])
    print ('정답', i[1])
    print ('출력', i[2])
print ('\n>부정 리뷰에 대한 예측:')
for i in neg[:5]:
    print ('\n입력', i[0])
    print ('정답', i[1])
    print ('출력', i[2])


>긍정 리뷰에 대한 예측:

입력 음악이 주가 된, 최고의 음악영화
정답 1
출력 ['1']

입력 괜찮네요오랜만포켓몬스터잼밌어요
정답 1
출력 ['1']

입력 청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다. 찰나의 아름다움을 잘 포착한 섬세하고 아름다운 수채화같은 퀴어영화이다.
정답 1
출력 ['1']

입력 눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다.
정답 1
출력 ['0']

입력 최고!!!!!!!!!!!!!!!!
정답 1
출력 ['1']

>부정 리뷰에 대한 예측:

입력 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
정답 0
출력 ['0']

입력 지루하지는 않은데 완전 막장임... 돈주고 보기에는....
정답 0
출력 ['0']

입력 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
정답 0
출력 ['0']

입력 진정한 쓰레기
정답 0
출력 ['0']

입력 한국독립영화의 한계 그렇게 아버지가 된다와 비교됨
정답 0
출력 ['1']


###KNN 모델 이용하기

In [ ]:
%%time
# KNN 코드
from sklearn.neighbors import KNeighborsClassifier
#neigh = KNeighborsClassifier(n_neighbors=5)
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X, train_Y)

print("Accuracy on training set: {:.3f}".format(neigh.score(train_X, train_Y)))
print("Accuracy on test set: {:.3f}".format(neigh.score(test_X, test_Y)))

Accuracy on training set: 0.732
Accuracy on test set: 0.700
CPU times: user 6.88 s, sys: 726 ms, total: 7.61 s
Wall time: 7.44 s


In [ ]:
pos = [ [test_docs_X[i], test_Y[i], neigh.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '1' ]
neg = [ [test_docs_X[i], test_Y[i], neigh.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '0' ]
#crr= []
#for i in range(len(test_Y)):
#    if test_Y[i] == '1':
#        crr.append([test_docs_X[i], test_Y[i], gnb_predict([test_X[i]]) ])
#print (crr)
print ('>긍정 리뷰에 대한 예측:')
for i in pos[:5]:
    print ('\n입력:', i[0])
    print ('정답:', i[1])
    print ('예측:', i[2])
print ('\n>부정 리뷰에 대한 예측:')
for i in neg[:5]:
    print ('\n입력:', i[0])
    print ('정답:', i[1])
    print ('예측:', i[2])


>긍정 리뷰에 대한 예측:

입력: 음악이 주가 된, 최고의 음악영화
정답: 1
예측: ['1']

입력: 괜찮네요오랜만포켓몬스터잼밌어요
정답: 1
예측: ['1']

입력: 청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다. 찰나의 아름다움을 잘 포착한 섬세하고 아름다운 수채화같은 퀴어영화이다.
정답: 1
예측: ['1']

입력: 눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다.
정답: 1
예측: ['1']

입력: 최고!!!!!!!!!!!!!!!!
정답: 1
예측: ['1']

>부정 리뷰에 대한 예측:

입력: 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
정답: 0
예측: ['1']

입력: 지루하지는 않은데 완전 막장임... 돈주고 보기에는....
정답: 0
예측: ['1']

입력: 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
정답: 0
예측: ['0']

입력: 진정한 쓰레기
정답: 0
예측: ['0']

입력: 한국독립영화의 한계 그렇게 아버지가 된다와 비교됨
정답: 0
예측: ['0']
